In [ ]:
%pip install gremlinpython

In [2]:
"""
Code to import connection data between elements and connection types into Cosmos DB (Graph API) from two JSON files:
• connection_types.json
• connections.json

Before running this script, create the Cosmos DB account with Gremlin and configure the connection parameters.
Database id: graphdb, Graph Id: graphdb, Partition Key: /pk

"""

import json
import os
import nest_asyncio
from dotenv import load_dotenv
from gremlin_python.driver import client, serializer
import networkx as nx
import matplotlib.pyplot as plt

# Function to execute a Gremlin query and obtain results
def execute_query(gremlin_client, query, bindings=None):
    if bindings is None:
        bindings = {}
    try:
        # The query is sent
        callback = gremlin_client.submit(query, bindings)
        result = callback.all().result()
        print("Query executed successfully:", result)
        return result
    except Exception as e:
        print("Error executing the query:", e)
        return None

# Asynchronous function to search for the component (by its name) and its relationships.
def search_component(gremlin_client, schema_id, comp_name):
    query = ("g.V().hasLabel('component')"
             ".has('pk', TextP.containing(schema_id))" #schema_id)"
             ".has('component_name', TextP.containing(comp_name))"
             ".as('component')"
             ".bothE().as('edge')"
             ".otherV().as('adjacent')"
             ".select('component','edge','adjacent')"
            )
    bindings = {"schema_id": schema_id, "comp_name": comp_name}
    try:
        results = execute_query(gremlin_client, query, bindings)
        return results
    except Exception as e:
        print("Error durante la búsqueda:", e)
        return None

# Asynchronous function to obtain the vertex of the component in case it has no relationships
def get_component_vertex(gremlin_client, schema_id, comp_name):
    query = ("g.V().hasLabel('component')"
             ".has('pk', schema_id)"
             ".has('component_name', comp_name)"
             ".elementMap()"
             )
    bindings = {"schema_id": schema_id, "comp_name": comp_name}
    try:
        results = execute_query(gremlin_client, query, bindings)
        return results[0] if results else None
    except Exception as e:
        print("Error al obtener el componente:", e)
        return None

# Function that interprets the string connection_style to assign a color and a line style.  
def parse_connection_style(style_str):  
    # Default values.  
    color = 'black'  
    line_style = 'solid'  
    if style_str:  
        lower = style_str.lower()  
        # Determine color (based on common keywords)  
        if 'blue' in lower:  
            color = 'blue'  
        elif 'red' in lower:  
            color = 'red'  
        elif 'green' in lower:  
            color = 'green'  
        elif 'pink' in lower:  
            color = 'pink'  
        elif 'brown' in lower:  
            color = 'brown'  
        # Determine the line style.  
        if 'dashed' in lower:  
            line_style = 'dashed'  
        elif 'dotted' in lower:  
            line_style = 'dotted'  
        else:  
            line_style = 'solid'  
    return color, line_style  

def plot_component_network(component_data, comp_name):
    """
    Draw the subgraph of the central component along with its relationships.

    The nodes are labeled with the component name (component_name).
    The edges display the value of connection_type and are drawn with the color and style
    according to the value of connection_style..  
    """  
    # A directed graph is created.  
    G = nx.DiGraph()  

    # Build the graph from the obtained data.  
    if component_data:  
        # The first record is taken as the central node.  
        central = component_data[0]['component']  
        central_id = central.get('id', comp_name)  

        central_label = central['properties'].get('component_name', [{}])[0].get('value')

        G.add_node(central_id, label=central_label)  

        for record in component_data:  
            edge = record.get('edge')  
            adjacent = record.get('adjacent')  
            if not (edge and adjacent):  
                continue  

            # The edge properties are obtained:  
            # connection_type will be used as the label on the edge.  
            conn_type = edge.get('properties', {}).get('connection_type', '')

            # connection_style will be used to determine the color and line type.  
            conn_style = edge.get('properties', {}).get('connection_style', '')

            edge_color, edge_line_style = parse_connection_style(conn_style)  
            # In this example, only connection_type will be used as the label.  
            edge_label = str(conn_type) if conn_type else ""  

            # The adjacent node information is obtained.
            adjacent_id = adjacent.get('id')
            # Get the value of component_name from the adjacent node.
            adjacent_label = adjacent.get('properties', {}).get('component_name', [f'Nodo {adjacent_id}'])[0]['value']
            G.add_node(adjacent_id, label=adjacent_label)  

            # The edge is added with additional attributes for drawing (color, style, and label).  
            G.add_edge(central_id, adjacent_id, label=edge_label,  
                    edge_color=edge_color, edge_style=edge_line_style)  
    else:  
        print("No relationships found for the component.")  
        return  

    # Generate a layout for the graph.  
    pos = nx.spring_layout(G)  
    node_labels = nx.get_node_attributes(G, 'label')  

    plt.figure(figsize=(10, 7))  
    nx.draw_networkx_nodes(G, pos, node_size=800, node_color='lightblue')  
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10)  

    # The edges are grouped by style and color to be drawn individually.  
    edge_groups = {}  
    for u, v, attr in G.edges(data=True):  
        key = (attr.get('edge_style', 'solid'), attr.get('edge_color', 'black'))  
        if key not in edge_groups:  
            edge_groups[key] = []  
        edge_groups[key].append((u, v))  

    # Draw the edges according to their groups (style and color).  
    for (style, color), edgelist in edge_groups.items():  
        nx.draw_networkx_edges(G, pos, edgelist=edgelist,  
                            arrowstyle='->', arrowsize=10,  
                            edge_color=color, style=style)  

    # Draw the edge labels (showing the connection_type).  
    edge_labels = nx.get_edge_attributes(G, 'label')  
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', font_size=8) #'red')  

    plt.title(f"Component Relationship Network: {comp_name}")  
    plt.axis('off')  
    plt.show()     
 

In [3]:
load_dotenv(override=True)

# o1 model configuration
cosmos_endpoint = os.getenv("COSMOS_ENDPOINT")
cosmos_db = os.getenv("COSMOS_DB")
cosmos_graph = os.getenv("COSMOS_GRAPH")
cosmos_partition_key = os.getenv("COSMOS_PARTITION_KEY")
cosmos_primary_key = os.getenv("PRIMARY_KEY")

# The Gremlin client for Cosmos DB is created
gremlin_client = client.Client(
    cosmos_endpoint,  
    "g",  
    username=f"/dbs/{cosmos_db}/colls/{cosmos_graph}",  
    password=cosmos_primary_key,  
    message_serializer=serializer.GraphSONSerializersV2d0()  
)
nest_asyncio.apply()

In [ ]:
# Search for the component with its relationships.
schema_id = '462-Piping'
comp_name = 'Piston Pump'

component_data = search_component(gremlin_client, schema_id, comp_name)  

# If the query did not return records, try to retrieve the central vertex only.  
if not component_data or len(component_data) == 0:  
    print("No relationships found. The component will be searched without connections..")  
    componente = get_component_vertex(gremlin_client, schema_id, comp_name)  
    if componente:  
        component_data = [{  
            "component": componente,  
            "edge": None,  
            "adjacent": None  
        }]  
    else:  
        print("The component does not exist in the specified schema.")  

#print("Query results:", json.dumps(component_data, indent=2))  
plot_component_network(component_data, comp_name)  

In [ ]:
# Cerrar la conexión  
gremlin_client.close()  


### Present the connection of every devices in a list

In [7]:
def present_connections(connections_filename):
    with open(connections_filename, "r", encoding="utf-8") as f:
        data = json.load(f)

    for component in data['connections']:
        print(f"Component: {component['component_name']} (Number: {component['component_number']})")
        print("Connected to:")
        for connection in component['connected_to']:
            for comp in data['connections']:
                if comp['component_number'] == connection['component_number']:
                    print(f"  - Component Name: {comp['component_name']}")
            print(f"  - Component Number: {connection['component_number']}")
            print(f"    Connection Style: {connection['connection_style']}")
            print(f"    Connection Type: {connection['connection_type']}")
        print()


In [ ]:
# Present the connections
print('--------------- 462-Piping ---------------')
present_connections('462-Piping/462-Piping_schema.json')